In [45]:
# @title Environment Setup
import os
import sys

# 1. *** FIX: Clear problematic environment variable for matplotlib ***
# This prevents the "ValueError: Key backend: 'module://matplotlib_inline.backend_inline'" error
if 'MPLBACKEND' in os.environ:
    del os.environ['MPLBACKEND']
    print("MPLBACKEND environment variable cleared.")

# 2. Clone the repository
!rm -rf FontDiffusion
!git clone https://github.com/dzungphieuluuky/FontDiffusion.git

!uv pip install --upgrade pip
!uv pip install -r FontDiffusion/requirements.txt
!uv pip install gdown
# 3. Install PyTorch 1.13
print("\n⬇️ Installing PyTorch 1.13 (Required for this model)...")
# Force reinstall torch 1.13 to match the model's training environment
# !uv pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 --extra-index-url https://download.pytorch.org/whl/cu117

# 4. Install other dependencies
print("\n⬇️ Installing Dependencies (Manually fixed)...")
# Install xformers compatible with Torch 1.13
!uv pip install xformers==0.0.16 -q

# Install Transformers & Diffusers
!uv pip install transformers==4.33.1 accelerate==0.23.0 diffusers==0.22.0
!uv pip install gradio==4.8.0 pyyaml pygame opencv-python info-nce-pytorch kornia
!uv pip install lpips scikit-image pytorch-fid
# -----------------------------------------------------------------
!sudo apt-get update && sudo apt-get install dos2unix
print("\n✅ Environment setup complete. You can now proceed to Block 2 (Inference).")

Cloning into 'FontDiffusion'...
remote: Enumerating objects: 15022, done.
remote: Counting objects: 100% (2835/2835), done.
remote: Compressing objects: 100% (2766/2766), done.
remote: Total 15022 (delta 76), reused 2822 (delta 67), pack-reused 12187 (from 3)
Receiving objects: 100% (15022/15022), 246.82 MiB | 30.03 MiB/s, done.
Resolving deltas: 100% (464/464), done.
Updating files: 100% (110/110), done.
Using Python 3.12.12 environment at: /usr
Resolved 1 package in 85ms
Audited 1 package in 0.16ms
Using Python 3.12.12 environment at: /usr
Resolved 116 packages in 453ms
  × Failed to build `tokenizers==0.13.3`
  ├─▶ The build backend returned an error
  ╰─▶ Call to `setuptools.build_meta.build_wheel` failed (exit status: 1)

      [stdout]
      running bdist_wheel
      running build
      running build_py
      copying py_src/tokenizers/__init__.py ->
      build/lib.linux-x86_64-cpython-312/tokenizers
      copying py_src/tokenizers/models/__init__.py ->
      build/lib.linux-x86_

In [46]:
import os
from IPython import get_ipython

def configure_environment_paths():
    """Detect environment and configure paths"""
    try:
        if "google.colab" in str(get_ipython()):
            print("✅ Environment: Google Colab")
            base_data_path = "/content/"
            base_output_path = "/content/"
            environment_name = "colab"
        elif os.environ.get("KAGGLE_KERNEL_RUN_TYPE"):
            print("✅ Environment: Kaggle")
            base_data_path = "/kaggle/input/"
            base_output_path = "/kaggle/working/"
            environment_name = "kaggle"
        else:
            print("⚠️ Environment: Local/Unknown")
            base_data_path = "./data/"
            base_output_path = "./output/"
            environment_name = "local"
    except NameError:
        print("⚠️ Non-interactive session. Using local paths.")
        base_data_path = "./data/"
        base_output_path = "./output/"
        environment_name = "local"

    os.makedirs(base_output_path, exist_ok=True)
    print(f"📂 Data Path: {base_data_path}")
    print(f"📦 Output Path: {base_output_path}")

    return base_data_path, base_output_path, environment_name


INPUT_PATH, OUTPUT_PATH, ENV_NAME = configure_environment_paths()

✅ Environment: Google Colab
📂 Data Path: /content/
📦 Output Path: /content/


In [47]:
import os
import wandb

if "colab" in ENV_NAME:
    from google.colab import userdata

    try:
        # Ensure 'WANDB_API_KEY' is the exact name in your Colab Secrets (the key icon)
        wandb_key = userdata.get("WANDB_API_KEY")
        wandb.login(key=wandb_key)
    except Exception as e:
        print(f"Could not retrieve W&B API key from Colab Secrets: {e}")

# 2. Check if running in Kaggle
elif "kaggle" in ENV_NAME:
    try:
        from kaggle_secrets import UserSecretsClient

        user_secrets = UserSecretsClient()
        wandb_key = user_secrets.get_secret("WANDB_API_KEY")
        wandb.login(key=wandb_key)
    except Exception as e:
        print(f"Could not retrieve W&B API key from Kaggle Secrets: {e}")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [48]:
import gdown

if not os.path.exists("ckpt"):
  url = "https://drive.google.com/drive/folders/12hfuZ9MQvXqcteNuz7JQ2B_mUcTr-5jZ"
  gdown.download_folder(url, quiet=True, use_cookies=False)

In [49]:
# @title Unzipping all archived files
import os
import glob
from zipfile import ZipFile

zip_file_paths = glob.glob(os.path.join(INPUT_PATH, '*.zip'))

if not zip_file_paths:
    print(f'No .zip files found in {INPUT_PATH}.')
else:
    for zip_file_path in zip_file_paths:
        if os.path.exists(zip_file_path):
            print(f'Unzipping {zip_file_path}...')
            !unzip -q -o {zip_file_path} -d ./
            print(f'Unzipping of {zip_file_path} complete.')
        else:
            print(f'Error: The file {zip_file_path} was not found (post-glob check).')

Unzipping /content/output_data.zip...
Unzipping of /content/output_data.zip complete.


In [50]:
# @title Checking checkpoint files (.pth)
import os
import time

CHECKPOINT_DIR = os.path.join(INPUT_PATH, "ckpt")
print(CHECKPOINT_DIR)
# Create the checkpoint directory
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
# Wait loop to check if files exist
required_files = ["unet.pth", "content_encoder.pth", "style_encoder.pth"]

while True:
    missing = [f for f in required_files if not os.path.exists(f"{CHECKPOINT_DIR}/{f}")]

    if not missing:
        print("\n✅ All weights found! You can proceed to the next step.")
        break
    else:
        print(f"Waiting for files... Missing: {missing}")
        print("Upload them to the 'ckpt' folder now.")
        time.sleep(10) # Checks every 10 seconds

/content/ckpt

✅ All weights found! You can proceed to the next step.


In [51]:
import pandas as pd
nom_tu_tao_300_df = pd.read_csv(f"{INPUT_PATH}/Ds_300_ChuNom_TuTao.csv")
nom_tu_tao = nom_tu_tao_300_df['word'].tolist()

with open(f"{OUTPUT_PATH}/nom_tu_tao.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(nom_tu_tao))

In [55]:
%cd {OUTPUT_PATH}
!python FontDiffusion/batch_sample_evaluate.py \
    --characters "nom_tu_tao.txt" \
    --style_images "FontDiffusion/DVSKTT_ref.jpg" \
    --ckpt_dir "ckpt/" \
    --output_dir "output_data" \
    --batch_size 8 \
    --ttf_path "FontDiffusion/fonts" \
    --num_inference_steps 20 \
    --guidance_scale 7.5 \
    --seed 42 \
    --fp16 \
    --enable_xformers \
    --channels_last \
    --compile \
    --use_wandb \
    --wandb_project "my-fontdiffuser" \
    --wandb_run_name ""

/content
pygame 2.6.1 (SDL 2.28.4, Python 3.12.12)
Hello from the pygame community. https://www.pygame.org/contribute.html

FONTDIFFUSER BATCH GENERATION & EVALUATION
Multi-Font Support
Successfully loaded 300 single characters.

Initializing font manager...

Loading 15 fonts from directory...
error: XDG_RUNTIME_DIR not set in the environment.
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unk

In [56]:
# @title Zipping the results folder
!zip -r {OUTPUT_PATH}/output_data.zip {OUTPUT_PATH}/output_data
print(f"Finish zipped the output data, ready for downloading")

updating: content//FontDiffusion/data_examples/train/ (stored 0%)
updating: content//FontDiffusion/data_examples/train/ContentImage/ (stored 0%)
updating: content//FontDiffusion/data_examples/train/ContentImage/镀.jpg (deflated 3%)
updating: content//FontDiffusion/data_examples/train/ContentImage/氮.jpg (deflated 3%)
updating: content//FontDiffusion/data_examples/train/ContentImage/舶.jpg (deflated 8%)
updating: content//FontDiffusion/data_examples/train/ContentImage/潮.jpg (deflated 5%)
updating: content//FontDiffusion/data_examples/train/TargetImage/ (stored 0%)
updating: content//FontDiffusion/data_examples/train/TargetImage/FZGuanJKSJW/ (stored 0%)
updating: content//FontDiffusion/data_examples/train/TargetImage/FZGuanJKSJW/FZGuanJKSJW+氮.jpg (deflated 2%)
updating: content//FontDiffusion/data_examples/train/TargetImage/FZGuanJKSJW/FZGuanJKSJW+舶.jpg (deflated 2%)
updating: content//FontDiffusion/data_examples/train/TargetImage/FZGuanJKSJW/FZGuanJKSJW+潮.jpg (deflated 2%)
updating: conten

In [57]:
# @title Happy Christmas✨
# !rm -r -f FontDiffusion